In [35]:
from submissions.ab_submission.object_detector import imageDataLoader, follicleClassifier, ObjectDetector
import os
import numpy as np

In [2]:
import glob
import pandas as pd
from matplotlib import pyplot as plt

train_files = glob.glob("./data/train/*.jpg")
test_files = glob.glob("./data/test/*.jpg")
train_label = pd.read_csv("./data/train/labels.csv")
test_label = pd.read_csv("./data/test/labels.csv")

In [3]:
model = ObjectDetector(ramp_mode=False)
model.load(boxPixelClassifier="./params/boxPixelClassifier_opencv.joblib", follicleClassifier="./params/follicleClassifier2.model")
model.fit(train_files, train_label)
#model.save(boxPixelClassifier="./params/boxPixelClassifier_opencv.joblib",follicleClassifier="./params/follicleClassifier2.model")

Model loaded from ./params/follicleClassifier2.model
Fitting boxPixelClassifier
Fitting follicleClassifier


C:\Users\4078182\Miniconda3\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
test_loader = imageDataLoader(test_files, test_label[["filename","xmin","xmax","ymin","ymax","label"]])
train_loader = imageDataLoader(train_files, train_label[["filename","xmin","xmax","ymin","ymax","label"]])

In [5]:
y_hat = model.predict(test_files)

In [31]:
from PIL import Image, ImageDraw, ImageFont

font_size = 60
if os.name != 'nt':
    font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", font_size)
else:
    font = ImageFont.truetype("C:/Windows/Fonts/Arial/arialbd.ttf", font_size)

def write_rectangle(image, preds, folder=None, filename=None):
    img = Image.fromarray(image)
    img_draw = ImageDraw.Draw(img)
    for pred in preds:
        x1, y1, x2, y2 = pred["bbox"]
        label = pred["class"]
        img_draw.rounded_rectangle(((x1, y1), (x2,y2)), fill=None, outline="black", width=5)
        img_draw.text((x1, y1-70), label, font=font, fill="black")

    if folder is not None and filename is not None:
        img.save(f"./data/{folder}/{filename}")
    
    return np.array(img)

In [36]:
i = 0
folder = "test_predicted"
for x in test_loader.get_samples():
    write_rectangle(x[0], y_hat[i], folder=folder, filename=test_loader.X_filenames[i])
    i += 1    

In [37]:
import os
from torch.utils.data import Dataset

In [39]:
class folliclesDataset(Dataset):
    """folliclesDataset
    
    This class provide a dataset for follicles algorithm training.
    The aim is to perform all the data transform and augmentation at the same place.
    This class provide an iterate, either it provide data in live, either it stores them in hard drive and provide them from memory 
    """

    def __init__ (self, imageLoader, local_path, verbose=True):
        """Parameters
            ----------
            image_loader: object from the image loader class
            data_augmentation: boolean, if True a data augmentation is performed
            local_path: str, local path for data storage which are kept in memory as picke in the local_path folder
            verbose: boolean, informations about current operations are displayed
        """

        # Storing the image loader
        self.imageLoader = imageLoader

        # Storing the parameters
        if local_path is not None and os.path.exists(local_path):
            self.local_path = local_path
            self.store_data = True
        else:
            raise Exception("The provided path doesn't exist.")

        # Generating data
        print("Generating data")

    def __len__(self):
        return 0